In [1]:
from selenium import webdriver
import re

EXE_PATH = "/Users/aprakash/Documents/personal/chromedriver"
driver = webdriver.Chrome(executable_path=EXE_PATH)

In [2]:
def goodreads_login(driver, username, password):
    """
    Login using credentials
    """
    driver.find_element_by_id("user_email").send_keys('arushi.prakash307@gmail.com')
    driver.find_element_by_id("user_password").send_keys("stupid@123")
    driver.find_element_by_name("next").click()
    
def go_to_page(driver, pagename):
    """
    Head to the webpage of choice
    """
    driver.get(pagename)
    
def click_till_disabled(driver, class_name):
    """
    Given that the page is open, finds the first element
    of class name and clicks on it, returns True.
    If element is not found, returns False.
    """
    import selenium.common.exceptions as selexcept 
    try:
        els = driver.find_element_by_class_name(class_name)
        if 'disabled' in els.get_attribute('class'):
            return False
        els.click()
    except selexcept.NoSuchElementException:
        return False
    return True


In [3]:
def open_new_tab(driver, page_name, tab_location):
    """
    Opens a new tab and switches focus to it, given the tab location
    """
    # Opened a new tab
    driver.execute_script('''window.open("about:blank", "_blank");''')
    # Switched focus to the new tab
    driver.switch_to_window(driver.window_handles[tab_location])
    # Opened the user profile
    driver.get(page_name)

In [4]:
def extract_userid(page_name):
    """
    Given a user_id link in GoodReads format, extract userid
    """
    userid = re.search(r'show/(.*?)-', page_name).group(1)
    return userid

In [13]:
def get_books_on_page(driver):
    """
    Given a books list page, return all the book names
    """
    books = []
    els = driver.find_elements_by_class_name('field.title')
    for el in els:
        try:
            books.append(el.find_element_by_class_name('value').text)
        except:
            pass
    return books

In [16]:
#Step by Step
# Step 1.  Go to the login page and sign in
driver = webdriver.Chrome(executable_path=EXE_PATH)
go_to_page(driver,
           pagename='https://www.goodreads.com/user/sign_in')
goodreads_login(driver,
                username="",
                password="")
# Step 2. Go to friends page to iterate over all friends
go_to_page(driver,
           pagename='https://www.goodreads.com/friend')
# Step 3. For each friend, extract userid and go to their read list
all_books = []
CLASS_NAME = 'next_page'
next_icon = True
while next_icon:
    # Analyze each friend in this page
    friends = driver.find_elements_by_class_name('userLink')
    for friend in friends:
        href = friend.get_attribute('href')
        userid = extract_userid(href)
        new_link = "https://www.goodreads.com/review/list/{}?shelf=read".format(userid)
        # Open new tab for each friend's read list
        open_new_tab(driver, new_link, 1)
        # Get the first twenty books
        books = get_books_on_page(driver)
        all_books.extend(books)
        # Close the tab for friends reading list
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
    # Go to next page if it exists
    next_icon = click_till_disabled(driver, CLASS_NAME)
# Sample page: https://www.goodreads.com/review/list/63180448?shelf=read
# Step 4. For the last 20 books in their list, find the names and find genre of the books

/Users/aprakash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  
/Users/aprakash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: use driver.switch_to.window instead


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=63370): Max retries exceeded with url: /session/58e74f9112766196732ebe7e9b652394/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x106c3dc88>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [17]:
all_books

['Bayside Opposites (The Hunters, #10)',
 "It's Not What It Looks Like",
 'Christmas on Candy Cane Lane (Life in Icicle Falls Book 8)',
 'Carve the Mark (Carve the Mark, #1)',
 'Shrill: Notes from a Loud Woman',
 'How to Speak Dragonese (How to Train Your Dragon, #3)',
 'Kidnapped (Everyday Heroes #4)',
 'How to Be a Pirate (How to Train Your Dragon, #2)',
 'Trapped (Everyday Heroes Book 3)',
 'Obsessed (Everyday Heroes #2)',
 'Hunted (Everyday Heroes #1)',
 'Running Home (Second Chances, #1)',
 'Fallen Witch (Spellbound Murder, #3)',
 'Grim Magic (Spellbound Murder, #2)',
 'Oath Bound (Spellbound Murder, #1)',
 'Kitty Confidential (Pet Whisperer PI #1)',
 'Last Man Standing',
 'Long Road to Mercy (Atlee Pine, #1)',
 'Between the Reins (Gold Valley Romance #4)',
 'Through the Mist (Gold Valley Romance #3)',
 'The Power of Habit: Why We Do What We Do and How to Change',
 'Here, There and Everywhere: Best-Loved Stories of Sudha Murthy',
 'Forever . . .',
 '2 States: The Story of My Marri

In [18]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [19]:
hist_, bins_ = np.histogram(all_books)

TypeError: cannot perform reduce with flexible type